# Clustering TCR timecourse data

This notebook will explore clustering the TCR data from the Adaptive Biotechnology time series. There are two published clustering algorithms, which appear to be highly similar.

One is from Mark Davis's group at Stanford called [Gliph](https://github.com/immunoengineer/gliph). The other is from Paul Thomas's group at St Jude Children’s Research Hospital called [tcr-dist](https://github.com/phbradley/tcr-dist).

These algorithms are quite slow, and are designed for 100-1000 TCRs.

## Working with Gliph

In [ ]:
import matplotlib.pyplot as plt, pandas as pd, numpy as np
import seaborn as sns, networkx as nx, os
from collections import Counter

In [ ]:
def gliph_format(adaptive_table, metadata):
    """
    convert tables from Adaptive format to gliph format
    
    The required columns are the CDR3 aa sequence, the V and J allele,
    And the patient identifier (or sample).
    
    Let's first try the column "PatientCounts" as the individual
    """
    gliph_table = adaptive_table[['aminoAcid', 'v', 'j']]
    gliph_table['PatientCounts'] = adaptive_table['sample'].apply(lambda x: metadata.set_index('sample_short').to_dict()['individual'][x])
    
    return gliph_table

In [ ]:
timecourse_aa = pd.read_csv('all_subjects_timecourse_w_nuc_aa.tsv.gz', 
                            sep='\t', compression='gzip')
metadata = pd.read_csv('tcrb_timecourse_metadata.txt', sep='\t')
epi_occ = pd.read_csv('tcr_epitopes_occurence_all.tsv.gz',
                                sep='\t', compression='gzip')

In [ ]:
#Let's first look at persistant TCRs from subject01 from sample s1_110316_PBMC
timecourse_aa_sub1 = timecourse_aa.loc[timecourse_aa['sample'] == 's1_110316_PBMC']

pers_tcrs_sub1 = epi_occ.loc[epi_occ['num_occ_in_sub1_pbmc'] == 8]['tcr_epitope'].values

#now get timecourse with only persistent tcrs
timecourse_pers_sub1 = timecourse_aa_sub1.loc[timecourse_aa_sub1['tcr_epitope'].isin(pers_tcrs_sub1)]

#get final table
timecourse_pers_sub1_gliph = gliph_format(timecourse_pers_sub1, metadata)

#write to file
timecourse_pers_sub1_gliph.to_csv('s1_111014_PBMC_pers_gliphtest.tsv',
                                 sep='\t', index=False)

In [ ]:
#after running the gliph pipeline, let's examine the results
clone_ntwk = pd.read_csv('s1_111014_PBMC_pers_gliphtest.tsv-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])

conv_groups = pd.read_csv('subject01_randomtcrs_7012-convergence-groups.txt',
                         sep='\t', header=None, names=['group_size',
                                                      'group_name',
                                                      'group_members'])

motifs = pd.read_csv('s1_111014_PBMC_pers_gliphtest.tsv-kmer_resample_1000_minp0.001_ove10.txt',
                    sep='\t')

kmer_resample = pd.read_csv('s1_111014_PBMC_pers_gliphtest.tsv-kmer_resample_1000_log.txt',
                           sep=' ')

The file clone network is 3 columns, with each row representing a pair of cdr3s that were significantly clustered together. The first two are lists of cdr3 regions that are "linked" or clustered together. The final columns is the type of convergence that resulted in the link: global, local, or singleton. Singletons are clones that have no links. Local are links based of off amino acid motifs or kmers. Global are links based on global similarity.

The congergence-groups file is the same information but in a different format. This file also has 3 columns, with each row representing a cluster of cdr3s. The first column is the number of cdr3s in that cluster. The second column is the name of the cluster. The third column is the membership of that cluster, separated by spaces.

The ove10 file provides amino acid motifs or kmers that were deemed significantly enriched in the provided tcr set. 'Motif' is the amino acid motif. 'Counts' is the number of occurences in the provided TCR set. 'avgRef' is the average number of occurences in the subsampled reference sets. 'topRef' is the maximum number of occurences in the subsamples reference sets. OvE is something close (but not exactly) the fold-enrichment of the provided TCR set versus the resampled reference sets. p-value is the p-value for this motif.

The kmer_resample file is the results from the N kmer resamples of the reference data set. the "Discovery" row is the results from the provided TCR set, and then each subsequent row is the results from a simulated resampling.

Looking at the data, 5.5% of the TCRs clustered with at least one other TCR.

Clustering questions:

1. Do persitent TCRs cluster to a greater or lesser extent than other TCRs?
2. Do public TCRs cluster to a greater or lesser extent?
3. Do both public and persistent TCRs cluster more or less across individuals?


In [ ]:
#first test to see how greatly the --global_vgene=0 flag changes results

So the --global_vgene=1 flag doesn't greatly change the results. The difference of using the flag is about the same as doing the analysis again with no flag (i.e. the difference is within the sampling noise). So let's roll with the -vgene flag because it's suggested and why not?

In [ ]:
#check on a single table with persistent TCRs from all individuals
sub1_pers = epi_occ.loc[epi_occ['num_occ_in_sub1_pbmc'] == 8]['tcr_epitope']
sub2_pers = epi_occ.loc[epi_occ['num_occ_in_sub2_pbmc'] == 8]['tcr_epitope']
sub3_pers = epi_occ.loc[epi_occ['num_occ_in_sub3_pbmc'] == 8]['tcr_epitope']

#Get only one time point of data, arbitrarily choose the first
pers_sub1_df = timecourse_aa.loc[timecourse_aa['sample'] =='s1_110316_PBMC']
pers_sub2_df = timecourse_aa.loc[timecourse_aa['sample'] =='s2_110317_PBMC']
pers_sub3_df = timecourse_aa.loc[timecourse_aa['sample'] =='s3_110316_PBMC']

#filter for persistent TCRs
pers_sub1_df = pers_sub1_df.loc[pers_sub1_df['tcr_epitope'].isin(sub1_pers)]
pers_sub2_df = pers_sub2_df.loc[pers_sub2_df['tcr_epitope'].isin(sub2_pers)]
pers_sub3_df = pers_sub3_df.loc[pers_sub3_df['tcr_epitope'].isin(sub3_pers)]

#add individual
pers_sub1_df['PatientCounts'] = 'subject01'
pers_sub2_df['PatientCounts'] = 'subject02'
pers_sub3_df['PatientCounts'] = 'subject03'

#combine and write to file
pers_df = pd.concat([pers_sub1_df, pers_sub2_df, pers_sub3_df])

pers_gliph = pers_df[['aminoAcid', 'v', 'j', 'PatientCounts']]
pers_gliph = pers_gliph.rename(columns={'aminoAcid': 'CDR3b', 'v': 'TRBV',
                                       'j': 'TRBJ'}) #rename columns
pers_gliph.to_csv('gliph/pers_tcrs_PBMC_allsub.txt', sep='\t',
                 index=False)

In [ ]:
#Do a preliminary test of random sets of 7012 TCRs (# of persistent TCRs in subject01)
#have to consider from what TCRs to pick and whether to weight the TCRs.
#without weighting, one risks picking too many TCRs that might be errors

#first get only tcrs from subject01
epi_occ_sub1 = epi_occ.loc[epi_occ['num_occ_in_sub1_pbmc'] > 0]

#pick them randomly from all TCRs, pick 100 sets
tcrs_rand1 = [np.random.choice(epi_occ_sub1['tcr_epitope'].values, 
                               size=7012) for x in range(100)]

#pick them randomly from nonpersistent TCRs
epi_occ1_nonpers = epi_occ_sub1.loc[(epi_occ_sub1['num_occ_in_sub1_pbmc'] != 8)]
tcrs_rand_nonpers = [np.random.choice(epi_occ1_nonpers['tcr_epitope'].values, 
                                      size=7012) for x in range(100)]

#pick them randomly from each category of occurence
tcr_sets = []
for x in [1, 2, 3, 4, 5, 6, 7]:
    tmp = epi_occ_sub1.loc[epi_occ['num_occ_in_sub1_pbmc'] == x]
    tcr_sets.append([np.random.choice(tmp['tcr_epitope'].values, 
                                      size=7012) for x in range(100)])
    
#write these outputs to a table
timecourse_aa.loc[timecourse_aa['sample'] =='s1_110316_PBMC']

In [ ]:
def subset_table_to_gliph(df, tcrs, patient_counts='unknown', output_file='gliph.txt'):
    """
    With a df, subset the df to only include the TCRs in the list tcrs.
    
    Then output a gliph table to a file
    """
    df = df.loc[df['tcr_epitope'].isin(tcrs)]
    df = df[['aminoAcid', 'v', 'j']]
    df = df.drop_duplicates()
    df['PatientCounts'] = patient_counts
    df = df.rename(columns={'aminoAcid': 'CDR3b', 'v': 'TRBV', 'j': 'TRBJ'})
    df.to_csv(output_file, sep='\t', index=False)

In [ ]:
conv_groups_pers = pd.read_csv('s1_111014_PBMC_pers_JL272017_vgene/s1_111014_PBMC_pers.tsv-convergence-groups.txt',
                         sep='\t', header=None, names=['group_size',
                                                      'group_name',
                                                      'group_members'])
pers_tcrs = Counter(conv_groups_pers['group_size'])
clone_ntwk_pers = pd.read_csv('s1_111014_PBMC_pers_JL272017_vgene/s1_111014_PBMC_pers.tsv-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])
pers_edges = Counter(clone_ntwk_pers['convergence_type'])

conv_groups1 = pd.read_csv('subject01_occ1_tcrs_7012-convergence-groups.txt',
                         sep='\t', header=None, names=['group_size',
                                                      'group_name',
                                                      'group_members'])
occ1_tcrs = Counter(conv_groups1['group_size'])
clone_ntwk_occ1 = pd.read_csv('subject01_occ1_tcrs_7012-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])
occ1_edges = Counter(clone_ntwk_occ1['convergence_type'])

conv_groups2 = pd.read_csv('subject01_occ2_tcrs_7012-convergence-groups.txt',
                         sep='\t', header=None, names=['group_size',
                                                      'group_name',
                                                      'group_members'])
occ2_tcrs = Counter(conv_groups2['group_size'])
clone_ntwk_occ2 = pd.read_csv('subject01_occ2_tcrs_7012-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])
occ2_edges = Counter(clone_ntwk_occ2['convergence_type'])

clone_ntwk_occ3 = pd.read_csv('subject01_occ3_tcrs_7012-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])
occ3_edges = Counter(clone_ntwk_occ3['convergence_type'])

clone_ntwk_occ4 = pd.read_csv('subject01_occ4_tcrs_7012-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])
occ4_edges = Counter(clone_ntwk_occ4['convergence_type'])

clone_ntwk_occ5 = pd.read_csv('subject01_occ5_tcrs_7012-clone-network.txt',
                        sep='\t', header=None, names=['cdr3_1', 
                                                      'cdr3_2', 
                                                      'convergence_type'])
occ5_edges = Counter(clone_ntwk_occ5['convergence_type'])

From looking at these broadly, my hunch is that clustering is signifcantly less for TCRs that only occur once or potentially Naive T cells, and that memory T cells or T cells that occur more than once have a greater tendency to cluster. If you look at the % TCRs that are singletons, the differece is slight (96-97% versus 91-94%), but there might be better ways to look at it. For example, the number of global convergences is 5.5X in persistent TCRs versus TCRs only in a single sample.

ALSO be careful when picking random sets of TCRs. There are more persistent TCRs than TCRs in 5, 6, or 7 samples.

In [ ]:
g = nx.Graph()
with open('s1_111014_PBMC_pers_JL272017_vgene/s1_111014_PBMC_pers.tsv-clone-network.txt', 
          'r+') as network_file:
    for line in network_file:
        line_split = line.rstrip().split('\t')
        if line_split != 'singleton':
            g.add_edge(line_split[0], line_split[1])

nx.average_clustering(g)

h = nx.Graph()
with open('subject01_occ1_tcrs_7012-clone-network.txt', 
          'r+') as network_file:
    for line in network_file:
        line_split = line.rstrip().split('\t')
        if line_split != 'singleton':
            h.add_edge(line_split[0], line_split[1])

nx.average_clustering(h)

i = nx.Graph()
with open('subject01_occ2_tcrs_7012-clone-network.txt', 
          'r+') as network_file:
    for line in network_file:
        line_split = line.rstrip().split('\t')
        if line_split != 'singleton':
            i.add_edge(line_split[0], line_split[1])

nx.average_clustering(i)

j = nx.Graph()
with open('subject01_occ3_tcrs_7012-clone-network.txt', 
          'r+') as network_file:
    for line in network_file:
        line_split = line.rstrip().split('\t')
        if line_split != 'singleton':
            j.add_edge(line_split[0], line_split[1])

nx.average_clustering(j)